### Import data from the new DAVIS

In [ ]:
# First, import packages

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings; warnings.simplefilter("ignore")

In [ ]:
# Read data
file = "24-25_02_2025.txt"                                  # insert here path (if not in the same folder of the notebook) and name of the file
delimiter = '\t'
df = pd.read_table(file, delimiter=delimiter, header=[0,1])

# Display the DataFrame
df

Unnamed: 0_level_0 Unnamed: 1_level_0  Temp    Hi   Low Out   Dew  Wind  \
                 Date               Time   Out  Temp  Temp Hum   Pt. Speed   
0            24/02/25               0:30  10.1  10.5  10.1  84   7.5   0.4   
1            24/02/25               1:00   9.9  10.1   9.9  85   7.5   0.4   
2            24/02/25               1:30   9.8   9.9   9.8  85   7.4   0.9   
3            24/02/25               2:00   9.8   9.8   9.7  86   7.5   0.4   
4            24/02/25               2:30   9.7   9.8   9.7  85   7.3   0.9   
..                ...                ...   ...   ...   ...  ..   ...   ...   
66           25/02/25              14:00  11.7  11.7  10.9  91  10.2   1.3   
67           25/02/25              14:30  12.2  12.2  11.6  90  10.6   0.9   
68           25/02/25              15:00  12.3  12.4  12.1  90  10.7   0.9   
69           25/02/25              15:30  12.3  12.3  12.2  89  10.5   0.9   
70           25/02/25              16:00  12.4  12.5  12.3  89  10.6   0.4   

               ...  In  In               In Air Unnamed: 33_level_0 Wind      \
    Dir   Run  ... Hum  Dew  Heat   EMC Density                 ET  Samp Tx    
0    NW  0.80  ...  37  5.9  19.7  7.35  1.2090                0.00  136   1   
1    NW  0.80  ...  38  6.2  19.6  7.55  1.2091                0.00  194   1   
2    NW  1.61  ...  38  6.3  19.7  7.55  1.2085                0.00  158   1   
3    NW  0.80  ...  38  6.2  19.6  7.55  1.2085                0.00  188   1   
4    NW  1.61  ...  38  6.2  19.6  7.55  1.2088                0.00  171   1   
..  ...   ...  ...  ..  ...   ...   ...     ...                 ...  ...  ..   
66  ESE  2.41  ...  49  9.1  19.3  9.09  1.1973                0.05  631   1   
67  ESE  1.61  ...  49  9.1  19.3  9.08  1.1969                0.00  640   1   
68   NE  1.61  ...  49  9.1  19.3  9.09  1.1970                0.05  610   1   
69   NE  1.61  ...  50  9.4  19.4  9.28  1.1961                0.00  588   1   
70   NW  0.80  ...  50  9.4  19.4  9.28  1.1961                0.03  614   1   

     ISS  Arc.  
   Recept Int.  
0    19.9   30  
1    28.4   30  
2    23.1   30  
3    27.5   30  
4    25.0   30  
..    ...  ...  
66   92.3   30  
67   93.6   30  
68   89.2   30  
69   86.0   30  
70   89.8   30  

[71 rows x 38 columns]

In [ ]:
df.columns = [' '.join(col) for col in df.columns]                  # Header is divided into two rows.  Merge them to have it on just one row.

new_col_names = {df.columns[0]:'Date', df.columns[1]:'Time', df.columns[16]:'Bar', df.columns[17]:'Rain',df.columns[33]:'ET'}
df = df.rename(columns=new_col_names)                               # Rename the titles with "unnamed".

df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y').dt.strftime('%Y-%m-%d')       # Date is in DD/MM/YY format, we need it in YYYY-MM-DD format.

df['Date'] = df['Date'].str.replace('-', '')                        # Remove "-" in dates and substitute ":" in times with "."
df['Time'] = df['Time'].str.replace(':', '.')

# Merge Date and Time columns into one named Datetime using the datetime module that can be interrogated.

day = df['Date'].values
hour = df['Time'].values
date_final = np.array([datetime.strptime(data + ' ' + ora, "%Y%m%d %H.%M") for data, ora in zip(day, hour)])
df.insert(2,'Datetime',date_final)
df.drop(df.columns[:2],axis=1, inplace=True)

# Identify numeric columns before conversion
non_numeric_cols = ['Wind Dir']  # Add other non-numeric columns if necessary

# Convert only numeric columns
df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda col: pd.to_numeric(col, errors='coerce') if col.name not in non_numeric_cols else col)


# Show processed dataframe
df

,Datetime,Temp Out,Hi Temp,Low Temp,Out Hum,Dew Pt.,Wind Speed,Wind Dir,Wind Run,Hi Speed,...,In Hum,In Dew,In Heat,In EMC,In Air Density,ET,Wind Samp,Wind Tx,ISS Recept,Arc. Int.
0,2025-02-24 00:30:00,10.1,10.5,10.1,84.0,7.5,0.4,NW,0.80,1.3,...,37,5.9,19.7,7.35,1.2090,0.00,136,1,19.9,30
1,2025-02-24 01:00:00,9.9,10.1,9.9,85.0,7.5,0.4,NW,0.80,0.9,...,38,6.2,19.6,7.55,1.2091,0.00,194,1,28.4,30
2,2025-02-24 01:30:00,9.8,9.9,9.8,85.0,7.4,0.9,NW,1.61,1.8,...,38,6.3,19.7,7.55,1.2085,0.00,158,1,23.1,30
3,2025-02-24 02:00:00,9.8,9.8,9.7,86.0,7.5,0.4,NW,0.80,1.3,...,38,6.2,19.6,7.55,1.2085,0.00,188,1,27.5,30
4,2025-02-24 02:30:00,9.7,9.8,9.7,85.0,7.3,0.9,NW,1.61,2.2,...,38,6.2,19.6,7.55,1.2088,0.00,171,1,25.0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,2025-02-25 14:00:00,11.7,11.7,10.9,91.0,10.2,1.3,ESE,2.41,3.1,...,49,9.1,19.3,9.09,1.1973,0.05,631,1,92.3,30
67,2025-02-25 14:30:00,12.2,12.2,11.6,90.0,10.6,0.9,ESE,1.61,3.1,...,49,9.1,19.3,9.08,1.1969,0.00,640,1,93.6,30
68,2025-02-25 15:00:00,12.3,12.4,12.1,90.0,10.7,0.9,NE,1.61,2.7,...,49,9.1,19.3,9.09,1.1970,0.05,610,1,89.2,30
69,2025-02-25 15:30:00,12.3,12.3,12.2,89.0,10.5,0.9,NE,1.61,2.7,...,50,9.4,19.4,9.28,1.1961,0.00,588,1,86.0,30
